### Aggregation of federated VUS analyses.
This notebook aggregates the results from executing the VUS analysis on three datasets carried out remotely on different platforms.

See the accompanying notebooks in this folder for the individual analyses.

The following are the DRS ids for the analysis results in the three different systems where they were conducted.

In [1]:
resultsFiles = {'osteosarcoma':'sbcav:619d69cd769fa54f1c4140e4',
               'copdgene': 'sbbdc:619d459b35d6fb33b51ec7e7'
               }

In [2]:
from fasp.loc import DRSMetaResolver
drsClient = DRSMetaResolver()

Invalid access token in /Users/forei/.keys/anvil_credentials.json
Searching the GA4GH registry for org.ga4gh:drs services
GA4GH registry unavailable, cannot get registered DRS services.
Continuing with locally known DRS services.


In [3]:
import requests
import os
def download(url, source):
    response = requests.get(url)
    data = response.json()
    # flatten cooccurrence output
    flat_vus = []
    for k, v in data['cooccurring vus'].items():
        pathogenic_count = len(v['pathogenic variants'])
        ## this is a pythonic way of merging dicts - it is cryptic
        z = {**{"source":source},**{"vus":k}, **v['likelihood data'], **v['allele frequencies'], **{"no_pathogenic_coocurrs":pathogenic_count}}
        flat_vus.append(z)
    return flat_vus

In [4]:
flat_vus = []
for source, drsCurie in resultsFiles.items():
    url = drsClient.getAccessURL(drsCurie,'s3')
    flat_vus = flat_vus + download(url,source)

# turn the array of dicts into a data frame    
import pandas as pd
flat_df = pd.DataFrame(flat_vus)
flat_df

/Users/forei/imfenvs/fasp/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,source,vus,p1,p2,n,k,likelihood,maxPop,maxPopFreq,cohortFreq,no_pathogenic_coocurrs
0,osteosarcoma,"('13', 32377590, 'CAAA', 'CAA')",0.045455,0.001,26,1,0.068651,NaN,NaN,0.787879,1
1,osteosarcoma,"('13', 32399885, 'C', 'A')",0.045455,0.001,33,3,0.000042,NaN,NaN,1.000000,3
2,osteosarcoma,"('13', 32400151, 'T', 'A')",0.045455,0.001,31,3,0.000038,NaN,NaN,0.939394,3
3,copdgene,"('13', 32342270, 'CA', 'C')",0.001077,0.001,5000,1,1.363553,NaN,NaN,0.489428,1
4,copdgene,"('13', 32344166, 'GAA', 'G')",0.001077,0.001,4192,1,1.281487,NaN,NaN,0.410337,1
5,copdgene,"('13', 32365109, 'C', 'CT')",0.001077,0.001,3444,1,1.209925,NaN,NaN,0.337118,1
6,copdgene,"('13', 32368001, 'CTTTTTTTTTT', 'C')",0.001077,0.001,6704,2,1.443371,NaN,NaN,0.656226,2
7,copdgene,"('13', 32392589, 'CAA', 'C')",0.001077,0.001,5432,2,1.309001,NaN,NaN,0.531715,2
8,copdgene,"('13', 32399786, 'C', 'CT')",0.001077,0.001,4495,1,1.311667,NaN,NaN,0.439996,1
9,copdgene,"('13', 32399885, 'C', 'A')",0.001077,0.001,10216,28,0.275916,NaN,NaN,1.000000,22


In [5]:
merged_df = flat_df.pivot(index='vus', columns='source', values=['n','cohortFreq','no_pathogenic_coocurrs'])

In [6]:
pd.set_option('display.max_rows',100)
merged_df

n              cohortFreq  \
source                               copdgene osteosarcoma   copdgene   
vus                                                                     
('13', 32342270, 'CA', 'C')            5000.0          NaN   0.489428   
('13', 32344166, 'GAA', 'G')           4192.0          NaN   0.410337   
('13', 32349814, 'C', 'CA')            6475.0          NaN   0.633810   
('13', 32353467, 'C', 'CAGAG')         4156.0          NaN   0.406813   
('13', 32353470, 'A', 'C')             4158.0          NaN   0.407009   
('13', 32353471, 'AAGC', 'A')          4157.0          NaN   0.406911   
('13', 32359222, 'GA', 'G')            7084.0          NaN   0.693422   
('13', 32359222, 'GAA', 'G')           4411.0          NaN   0.431774   
('13', 32365109, 'C', 'CT')            3444.0          NaN   0.337118   
('13', 32365109, 'CT', 'C')            3739.0          NaN   0.365995   
('13', 32368001, 'CTTTTTTTTTT', 'C')   6704.0          NaN   0.656226   
('13', 32368798, 'GT', 'G')            7722.0          NaN   0.755873   
('13', 32377590, 'CAAA', 'CAA')           NaN         26.0        NaN   
('13', 32388121, 'CT', 'C')            4800.0          NaN   0.469851   
('13', 32388666, 'C', 'CTT')           5162.0          NaN   0.505286   
('13', 32388666, 'C', 'CTTT')          4306.0          NaN   0.421496   
('13', 32392589, 'CAA', 'C')           5432.0          NaN   0.531715   
('13', 32399786, 'C', 'CT')            4495.0          NaN   0.439996   
('13', 32399885, 'C', 'A')            10216.0         33.0   1.000000   
('13', 32400151, 'T', 'A')             9620.0         31.0   0.941660   

                                                  no_pathogenic_coocurrs  \
source                               osteosarcoma               copdgene   
vus                                                                        
('13', 32342270, 'CA', 'C')                   NaN                    1.0   
('13', 32344166, 'GAA', 'G')                  NaN                    1.0   
('13', 32349814, 'C', 'CA')                   NaN                    1.0   
('13', 32353467, 'C', 'CAGAG')                NaN                    1.0   
('13', 32353470, 'A', 'C')                    NaN                    1.0   
('13', 32353471, 'AAGC', 'A')                 NaN                    1.0   
('13', 32359222, 'GA', 'G')                   NaN                    1.0   
('13', 32359222, 'GAA', 'G')                  NaN                    1.0   
('13', 32365109, 'C', 'CT')                   NaN                    1.0   
('13', 32365109, 'CT', 'C')                   NaN                    1.0   
('13', 32368001, 'CTTTTTTTTTT', 'C')          NaN                    2.0   
('13', 32368798, 'GT', 'G')                   NaN                    2.0   
('13', 32377590, 'CAAA', 'CAA')          0.787879                    NaN   
('13', 32388121, 'CT', 'C')                   NaN                    3.0   
('13', 32388666, 'C', 'CTT')                  NaN                    2.0   
('13', 32388666, 'C', 'CTTT')                 NaN                    2.0   
('13', 32392589, 'CAA', 'C')                  NaN                    2.0   
('13', 32399786, 'C', 'CT')                   NaN                    1.0   
('13', 32399885, 'C', 'A')               1.000000                   22.0   
('13', 32400151, 'T', 'A')               0.939394                   13.0   

                                                   
source                               osteosarcoma  
vus                                                
('13', 32342270, 'CA', 'C')                   NaN  
('13', 32344166, 'GAA', 'G')                  NaN  
('13', 32349814, 'C', 'CA')                   NaN  
('13', 32353467, 'C', 'CAGAG')                NaN  
('13', 32353470, 'A', 'C')                    NaN  
('13', 32353471, 'AAGC', 'A')                 NaN  
('13', 32359222, 'GA', 'G')                   NaN  
('13', 32359222, 'GAA', 'G')                  NaN  
('13', 32365109, 'C', 'CT')                

Note, the TCGA locations are in GRCh37 coordinates. The cooccurrence code uses the location as the identifier of the vus. Need to match variants across reference genomes. 